In [1]:
pip install openpyxl


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pickle
import time
from datetime import datetime
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import openpyxl
def main():
    def slow_scroll(driver):
        """Simulate slower scrolling by pressing the Page Down key."""
        for _ in range(random.randint(2, 4)):  # Simulate a few scrolls
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(random.uniform(1, 2))  # Randomize wait time between scrolls

    def random_sleep(minimum=5, maximum=10):
        """Introduce random sleep intervals to mimic human behavior."""
        time.sleep(random.uniform(minimum, maximum))

    def save_to_excel(data, filename):
        """Save the collected data to an Excel file."""
        if os.path.exists(filename):
            df_existing = pd.read_excel(filename, engine='openpyxl')
            df_new = pd.DataFrame(data, columns=["Username", "Tweets", "Date", "Location"])
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined.to_excel(filename, index=False, engine='openpyxl')
        else:
            df = pd.DataFrame(data, columns=["Username", "Tweets", "Date", "Location"])
            df.to_excel(filename, index=False, engine='openpyxl')

    def save_last_tweet_date(date):
        with open("last_tweet_date.txt", "w") as file:
            file.write(date)

    def load_last_tweet_date():
        try:
            with open("last_tweet_date.txt", "r") as file:
                return file.read().strip()
        except FileNotFoundError:
            return None
        
    def convert_date_format(date_str):
        """Convert date from '2024-02-06T20:48:02.000Z' to '2024-02-06_20:48:02_UTC' format."""
        # Parse the datetime string to a datetime object
        date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        # Convert back to the desired string format without subtracting a day
        new_date_str = date_obj.strftime("%Y-%m-%d_%H:%M:%S_UTC")
        return new_date_str


    options = Options()
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("window_size=1280,800")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-save-password-bubble")


    # Initialize WebDriver
    driver = webdriver.Chrome(options=options)
    """ driver.get("https://accounts.google.com/v3/signin/identifier?hl=en_GB&ifkv=AXo7B7VGP4Y_gNfwPri72zV40Ii9kmgYbvLRXoOhOeBNkeBYcMPcPOX_Aolo1vK16FetaA4URMIfUA&flowName=GlifWebSignIn&flowEntry=ServiceLogin&dsh=S-1140670556%3A1692882589574310")
    #email: 
    email = 'chadtinderman@gmail.com'
    #password:
    password = 'Mamama00.'

    #LOGIN
    driver.find_element(By.XPATH,'//*[@id="identifierId"]').send_keys(email)
    time.sleep(3)
    driver.find_element(By.XPATH,'//*[@id="identifierNext"]/div/button/span').click()
    time.sleep(5)
    driver.find_element(By.XPATH,'//*[@id="password"]/div[1]/div/div[1]/input').send_keys(password)
    driver.find_element(By.XPATH,'//*[@id="passwordNext"]/div/button/span').click()
    time.sleep(5) """

    #GO TO TWITTER
    driver.get("https://twitter.com/")
    time.sleep(5)
    # Save cookies to a file after login
    """ with open("twitter_cookies.pkl", "wb") as cookies_file:
        pickle.dump(driver.get_cookies(), cookies_file) """

    # Load and add the saved cookies
    with open("twitter_cookies.pkl", "rb") as cookies_file:
        cookies = pickle.load(cookies_file)
        for cookie in cookies:
            driver.add_cookie(cookie)
    time.sleep(5)

    # After loading cookies, navigate to the site again to access it as a logged-in user
    driver.get("https://twitter.com")

    # Load the last tweet date
    last_saved_date = load_last_tweet_date()
    if last_saved_date:
        # Convert the last saved date to the required format
        last_saved_date = convert_date_format(last_saved_date)
        search_term = f'Milei since:2024-01-30_21:00:00_UTC until:{last_saved_date} lang:es'
    else:
        search_term = 'Milei since:2024-01-30_21:00:00_UTC until:2024-02-06_21:00:00_UTC lang:es'

    driver.get(f"https://twitter.com/search?q={search_term}&src=typed_query")
    random_sleep(2, 4)  # Wait for page to load

    user_tweets_count = {}
    tweets_data = []
    tweet_counter = 0

    try:
        latest_tweets_tab = driver.find_element(By.XPATH, '//span[contains(text(),"Latest")]')
        latest_tweets_tab.click()
        random_sleep(2, 4)  # Wait for the latest tweets to load
    except Exception as e:
        print("Could not switch to 'Latest' tweets tab:", e)
    last_tweet_date = None
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
        for tweet in tweets:
            username_element = tweet.find_element(By.XPATH, ".//span[contains(text(),'@')]")
            tweet_text_element = tweet.find_element(By.XPATH, ".//div[@data-testid='tweetText']")
            date_element = tweet.find_element(By.XPATH, ".//time")
            date_text = date_element.get_attribute('datetime') if date_element else "Not available"
            
            username = username_element.text
            tweet_text = tweet_text_element.text
            if user_tweets_count.get(username, 0) < 3:
                tweets_data.append({"Username": username, "Tweets": tweet_text, "Date": date_text})
                user_tweets_count[username] = user_tweets_count.get(username, 0) + 1
                last_tweet_date = date_text  # Update the last tweet's date
                tweet_counter += 1  # Increment the tweet counter

                # Save every 100 tweets
                if tweet_counter % 100 == 0:
                    print(f"Saving progress at {tweet_counter} tweets...")
                    save_to_excel(tweets_data, "tweets.xlsx")
                    tweets_data = []  # Clear the list to start collecting the next batch of tweets

        slow_scroll(driver)
        random_sleep(5, 10)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached the bottom of the page or no new tweets are loading.")
            break
        last_height = new_height

    # Save any remaining tweets after exiting the loop
    if tweets_data:  # If there are any tweets left in the list, save them
        save_to_excel(tweets_data, "tweets.xlsx")
        if last_tweet_date:  # Make sure there is a last tweet date to save
            save_last_tweet_date(last_tweet_date)

    driver.quit()
    print(f"Tweets saved to: {os.path.join(os.getcwd(), 'tweets.xlsx')}")
while True:
    main()
    time.sleep(10)  # Delay for 10 seconds or any suitable interval

Saving progress at 100 tweets...
Saving progress at 200 tweets...
Saving progress at 300 tweets...
Saving progress at 400 tweets...
Saving progress at 500 tweets...
Saving progress at 600 tweets...
Saving progress at 700 tweets...
Saving progress at 800 tweets...
Saving progress at 900 tweets...
Saving progress at 1000 tweets...
Saving progress at 1100 tweets...
Saving progress at 1200 tweets...
Saving progress at 1300 tweets...
Saving progress at 1400 tweets...
Saving progress at 1500 tweets...
Saving progress at 1600 tweets...
Saving progress at 1700 tweets...
Saving progress at 1800 tweets...
Saving progress at 1900 tweets...
Saving progress at 2000 tweets...
Saving progress at 2100 tweets...
Saving progress at 2200 tweets...
Saving progress at 2300 tweets...
Saving progress at 2400 tweets...
Saving progress at 2500 tweets...
Saving progress at 2600 tweets...
Saving progress at 2700 tweets...
Saving progress at 2800 tweets...
Saving progress at 2900 tweets...
Saving progress at 3000